In [3]:
# ~ IMPORTS ~
import qrcode
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from pyzbar.pyzbar import decode, ZBarSymbol
import cv2
import os
import ast

In [153]:
# ~ PARAMETERS ~

# Metadata for QR code

# Example dictionary
# qrMetadata = {
#     "user": "Zoe",
#     "date": "2/19/2020",
#     "experiment": "PI Stuff",
#     "mood": "stressed",
#     "volenum": "1234",
#     "location": "TL"
# }

# Dictionary of positional info
qrMetadata = {
    
    # Change based on where the QR will be placed.
    # Values: TL, TR, BL, BR - (Top Left, Top Right, Bottom Left, Bottom Right)
    "location": "TL",
    
    # Tuple of x,y coordinates in cm
    "coordinates": (0,0)
}

# Directory of supplementary info
qrSup = {
    # Who is conducting the experiment
    "user": "Dave",
    
    # Desired Date
    "date": "2/21/2020",
    
    # Experiment Title
    "experiment": "QR Code 4 Corners",
    
    # Change the color of the QR code
    "qrColor": "Black",

    # Change based on wether the QR is for position or vole
    # Values: Position, Vole
    "qrType": "Position",
    
    # Change based on the vole number
    "voleNum": "1234",
    
    # Directory used to save the QR code
    "fileDir": r'D:\Donaldson Lab\QRCodes',

    # Name of the individual QR code
    "fileName": r'\TL_Test',

    # Image type (ex. png, bmp, jpeg)
    "extension": "png"
}

In [154]:
# ~ QR ENCODER ~ 
# Passed a dictionary containing all of the neccessary metadata for the QR
# Doesn't return anything
def QREncoder(MetadataDict, SupDict):
    
    # Create qr code instance
    qrEncoder = qrcode.QRCode(
        version = 1,
        error_correction = qrcode.constants.ERROR_CORRECT_H,
        box_size = 10,
        border = 4,
    )

    # Add data
    qrEncoder.add_data(MetadataDict)
    qrEncoder.make(fit=True)

    # Create an image from the QR Code instance
    img = qrEncoder.make_image(fill_color= SupDict["qrColor"])
    
    # Create directory if it doesn't exist
    try:
        os.mkdir(SupDict["fileDir"])
    except:
        pass
    
    # Save QR to desired filepath
    img.save(SupDict["fileDir"] + SupDict["fileName"] + "." + SupDict["extension"])
    
    # Open the QR again, this time in color
    img = Image.open(SupDict["fileDir"] + SupDict["fileName"] + "." + SupDict["extension"]).convert("RGBA")

    # Get the width and height of the created QR
    width, height = img.size
   
    # Draw QR code 
    draw = ImageDraw.Draw(img)

    # If it is a positional QR code, then draw boxes
    if SupDict["qrType"] == "Position":
        # Draw red outter box
        draw.rectangle([(40,40),(109,109)], outline="red", width = 10)
        # Draw red inner box
        draw.rectangle([(60,60),(89,89)], fill="red")
    
        # Check which corner the QR is located, rotate point of interest to that location
        if MetadataDict["location"] == "TR":
            img = img.rotate(270)
        elif MetadataDict["location"] == "BL":
            img = img.rotate(90)
        elif MetadataDict["location"] == "BR":
            img = img.rotate(180)
        else:
            pass
    # If it is a vole QR code, then draw text showing the vole number
    elif SupDict["qrType"] == "Vole":
        # Declare fonts
        topTextFont = ImageFont.truetype("arial.ttf", 35)
        # Draw text that displays the vole number in black
        draw.text((0, 0), "Vole Number: %s" % SupDict["voleNum"], font=topTextFont, fill="black")
    else:
        pass

    # Save it to desired filepath, change the extension as needed:
    img.save(SupDict["fileDir"] + SupDict["fileName"] + "." + SupDict["extension"])

In [155]:
# ~ QR DECODER ~ 
# Passed a frame of a video using either cv2.imread or Image.open
# Adds a tuple of pixels where the QR location is.
# For example, location: TL => function adds the minX and maxY of the QR code to thats QR code's dictionary
# Returns a list of dictionaries containing any QR code metadata found in the frame.

#TODO: FIX THIS FUNCTION. ITS DOES NOT NORMALIZE AS IS. 
#      IT FINDS THE CORNER, IN PIXELS, OF THE QR CODES CORRESPONDING TO THEIR LOCATION KEY.
def QRDecoder(frame, SupDict):
    
    # Decode returns a list of objects
    # [ 
    #   data = b'...' => The QR metadata
    #   rect = Rectangle(...) => This is a rectangle object representing the size of the QR
    #   Polygon = [Point(...)...] => A Polygon object to show where the QR was located in the image.
    # ] 
    decoded = decode(frame, symbols=[ZBarSymbol.QRCODE])
    
    # List to store the dictionary values from the QR codes
    dicts = []
    
    # Add the QR dictionaries to the list
    for qr in decoded:
        # Convert B string info to dictionaries
        dict = ast.literal_eval(qr[0].decode('utf-8'))
        
        # Add location of POI to dictionary
        # TL -> Min X, Max Y
        # TR -> Max X, Max Y
        # BL -> Min X, Min Y
        # BR -> Max X, Min Y
        maxX = -1
        maxY = -1
        minX = 999999
        minY = 999999
        
        # Find the min and max points of the bounding box
        # Maybe a faster a way to do this?!? Max and Min acting weird on Y coordinate
        for pnt in qr[3]:
            if pnt[0] > maxX:
                maxX = pnt[0]
            if pnt[0] < minX:
                minX = pnt[0]
            if pnt[1] > maxY:
                maxY = pnt[1]
            if pnt[1] < minY:
                minY = pnt[1]
        
        # Assign the corresponding min and max values based on the corner.
        bbTL = (minX, maxY)
        bbTR = (maxX, maxY)
        bbBL = (minX, minY)
        bbBR = (maxX, minY)

        # Add bounding box location metric
        if SupDict.get("qrType") == "Position":
            if dict.get("location") == "TL":
                dict["bb"] = bbTL
            elif dict.get("location") == "TR":
                dict["bb"] = bbTR
            elif dict.get("location") == "BL":
                dict["bb"] = bbBL
            elif dict.get("location") == "BR":
                dict["bb"] = bbBR
            else:
                pass
        else:
            pass
        
        
        # Append the dictionary to the list
        dicts.append(dict)
        
        # INSERT MATH HERE TO CALCULATE THE EUCLIDIAN DISTANCE
        
    # Noticed that decode inserts items from bottom to top and right to left.
    # Reversing the list puts the top left QR code first
    dicts.reverse()
   
    # RETURN: List of dictonaries from all decoded QR codes.
    return dicts

In [156]:
QREncoder(qrMetadata, qrSup)
try:
    qrCodesFound = QRDecoder(cv2.imread(r'D:\Donaldson Lab\QRCodes\WIN_20200221_11_09_08_Pro.jpg'), qrSup)
    
    # TESTING: print the qr codes found
    for qr in qrCodesFound:
        print(qr)
except:
    print("Error")

{'location': 'TR', 'bb': (958, 140)}
{'location': 'BR', 'bb': (940, 592)}
